파일경로 및 이름만 신경써주면 작동합니다.

서울대 2층 24년

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import io

# ============================================================
# 0) 기본 설정 (1시간 기준)
# ============================================================
EMETER_DIR = Path("/home/hd/Desktop/LOG_EMETER_snu_02")   # TODO: 실제 폴더로 수정
OUT_CSV    = "./preprocessed_1h_master_emeter_20240501_1031_snu_02.csv"

start = pd.Timestamp("2024-05-01 00:00:00")
end   = pd.Timestamp("2024-10-31 23:00:00")
master_index = pd.date_range(start=start, end=end, freq="1H")

# 사용자 정의 휴일(날짜만)
HOLIDAY_LIST_TEXT_2025 = """
2024-01-01,월,신정,공휴일
2024-02-09,금,설날 연휴,공휴일
2024-02-10,토,설날,공휴일
2024-02-11,일,설날 연휴,공휴일
2024-02-12,월,대체공휴일,설날 대체공휴일
2024-03-01,금,삼일절,공휴일
2024-04-10,수,제22대 국회의원 선거,공휴일
2024-05-01,수,근로자의 날,기념일(일부 휴무)
2024-05-05,일,어린이날,공휴일
2024-05-06,월,대체공휴일,어린이날 대체공휴일
2024-05-15,수,부처님오신날,공휴일
2024-06-06,목,현충일,공휴일
2024-08-15,목,광복절,공휴일
2024-09-16,월,추석 연휴,공휴일
2024-09-17,화,추석,공휴일
2024-09-18,수,추석 연휴,공휴일
2024-10-01,화,국군의 날,임시공휴일(2024 한정)
2024-10-03,목,개천절,공휴일
2024-10-09,수,한글날,공휴일
2024-12-25,수,기독탄신일(성탄절),공휴일
""".strip()

def parse_holiday_dates(holiday_text: str) -> set:
    """
    holiday_text에서 첫 번째 컬럼(YYYY-MM-DD)만 파싱하여 date set으로 반환.
    """
    dates = []
    for line in holiday_text.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 1:
            continue
        d = pd.to_datetime(parts[0], errors="coerce")
        if pd.isna(d):
            continue
        dates.append(d.date())
    return set(dates)

HOLIDAY_DATES_2025 = parse_holiday_dates(HOLIDAY_LIST_TEXT_2025)



# ============================================================
# 1) 파일 이름에서 날짜(YYYYMMDD) 뽑기
# ============================================================
def extract_date_from_filename(filename: str) -> pd.Timestamp:
    m = re.search(r"(\d{8})", filename)
    if not m:
        raise ValueError(f"파일명에서 날짜(YYYYMMDD)를 찾을 수 없습니다: {filename}")
    return pd.to_datetime(m.group(1), format="%Y%m%d")

# ============================================================
# 2) Time 컬럼과 파일명 날짜로 datetime 만들기
# ============================================================
def make_datetime_from_time_and_filename(df: pd.DataFrame, file_path: Path) -> pd.DataFrame:
    if "Time" not in df.columns:
        raise ValueError(f"'Time' 컬럼이 없습니다. 파일: {file_path}")

    file_date = extract_date_from_filename(file_path.name)

    df = df.copy()
    df["datetime"] = pd.to_datetime(
        file_date.strftime("%Y-%m-%d") + " " + df["Time"].astype(str),
        errors="coerce"
    )
    df = df.dropna(subset=["datetime"])
    df = df.sort_values("datetime").set_index("datetime")
    return df

# ============================================================
# 3) NULL(\x00) 제거 후 read_csv
# ============================================================
def read_csv_remove_nulls(path: Path) -> pd.DataFrame:
    with open(path, "rb") as fh:
        raw = fh.read()
    if b"\x00" in raw:
        print(f"[WARN] NULL 바이트 감지 → 제거 후 로드: {path.name}")
        raw = raw.replace(b"\x00", b"")
    return pd.read_csv(io.BytesIO(raw))

# ============================================================
# 4) 폴더 안 CSV를 모두 읽어서 concat
# ============================================================
def load_all_csv_time_from_filename(folder: Path, pattern: str) -> pd.DataFrame | None:
    files = sorted(folder.glob(pattern))
    if not files:
        print(f"[WARN] {folder} 에 {pattern}에 해당하는 파일이 없습니다.")
        return None

    dfs = []
    for f in files:
        try:
            df_raw = read_csv_remove_nulls(f)
            df = make_datetime_from_time_and_filename(df_raw, f)
            df["src_file"] = f.name
            dfs.append(df)
        except Exception as e:
            print(f"[ERROR] {f} 로드 실패: {e}")

    if not dfs:
        return None

    return pd.concat(dfs, axis=0).sort_index()

# ============================================================
# 5) 시간 feature 생성 (1시간 기준)
#    - is_holiday
#    - day_cos/sin: 24-step
#    - week_cos/sin: 168-step
# ============================================================
def create_time_features_1h(index: pd.DatetimeIndex,
                            holiday_dates: set) -> pd.DataFrame:
    df_time = pd.DataFrame(index=index)

    # 토/일 여부: Mon=0 ... Sun=6
    is_weekend = (df_time.index.weekday >= 5)

    # 공휴일 리스트 포함 여부
    dates = df_time.index.date
    is_listed_holiday = pd.Series(dates, index=df_time.index).isin(holiday_dates).values

    # 최종 휴일 플래그
    df_time["is_holiday"] = (is_weekend | is_listed_holiday).astype(int)

    # 24-step cycle
    hour_of_day = df_time.index.hour.astype(int)  # 0~23
    df_time["day_sin"] = np.sin(2 * np.pi * hour_of_day / 24.0)
    df_time["day_cos"] = np.cos(2 * np.pi * hour_of_day / 24.0)

    # 168-step cycle (start 기준으로 시간 인덱스)
    k = np.arange(len(df_time), dtype=float)
    df_time["week_sin"] = np.sin(2 * np.pi * k / 168.0)
    df_time["week_cos"] = np.cos(2 * np.pi * k / 168.0)

    return df_time
# ============================================================
# 6) EMETER 전처리 (10분단위 c_real → 1시간 합)
#    - slot: ceil("1H") 사용 (창 끝 시각)
#    - slot별 c_real 합 = Power_1h
#    - 어떤 slot에 row가 없으면 Power_1h = 0
#
# 예)
#  target slot = 2025-10-07 12:00
#  emeter rows: 11:20:15, 11:30:15만 존재
#  => 둘 다 ceil -> 12:00 슬롯에 들어가므로 합쳐짐
# ============================================================
def preprocess_emeter_power_1h(df_emeter: pd.DataFrame,
                              freq: str = "1H",
                              power_col: str = "c_real") -> pd.DataFrame:
    # 컬럼명 대소문자/변형 대응
    cols_lower = {c.lower(): c for c in df_emeter.columns}
    if power_col.lower() not in cols_lower:
        raise ValueError(f"EMETER에 '{power_col}' 컬럼이 없습니다. 현재 컬럼: {df_emeter.columns.tolist()}")
    P_COL = cols_lower[power_col.lower()]

    df = df_emeter.copy()
    df[P_COL] = pd.to_numeric(df[P_COL], errors="coerce").fillna(0.0)

    # slot (창 끝)
    df["slot"] = df.index.ceil(freq)

    # slot별 합
    p_sum = df.groupby("slot")[P_COL].sum()

    # master timeline에 맞춰: 없는 slot은 0
    out = pd.DataFrame({"Power_1h": p_sum}).reindex(master_index).fillna(0.0)
    return out

# ============================================================
# 7) 전체 파이프라인
# ============================================================
def main():
    # 1) 시간 feature
    df_time = create_time_features_1h(master_index, holiday_dates=HOLIDAY_DATES_2025)

    # --------------------------------------------------------
    # 2) EMETER 로드 & 슬라이싱
    # --------------------------------------------------------
    # TODO: 파일 패턴을 실제 파일명에 맞게 수정 (예: "DBG_EMETER_*.csv")
    df_emeter_raw = load_all_csv_time_from_filename(EMETER_DIR, "*.csv")
    if df_emeter_raw is None:
        raise RuntimeError("EMETER 로그를 읽지 못했습니다. 경로/패턴을 확인하세요.")
    print("[INFO] EMETER raw shape:", df_emeter_raw.shape)

    raw_start = start - pd.Timedelta("1H")
    df_emeter_raw = df_emeter_raw.loc[(df_emeter_raw.index >= raw_start) & (df_emeter_raw.index <= end)]
    print("[INFO] EMETER clipped shape:", df_emeter_raw.shape)

    power_1h = preprocess_emeter_power_1h(df_emeter_raw, freq="1H", power_col="c_real")
    print("[INFO] EMETER 1H shape:", power_1h.shape)

    # --------------------------------------------------------
    # 3) merge (요청: day/week sincos + holiday + power)
    # --------------------------------------------------------
    df_all = df_time.join(power_1h, how="left")
    df_all["Power_1h"] = df_all["Power_1h"].fillna(0.0)

    print("[INFO] Final shape:", df_all.shape)
    print(df_all.head(3))
    print(df_all.tail(3))

    df_all.to_csv(OUT_CSV, index_label="datetime")
    print(f"[INFO] Saved → {OUT_CSV}")

if __name__ == "__main__":
    main()


서울대 2층 25년 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import io

# ============================================================
# 0) 기본 설정 (1시간 기준)
# ============================================================
EMETER_DIR = Path("/home/hd/Desktop/LOG_EMETER_snu_02")   # TODO: 실제 폴더로 수정
OUT_CSV    = "./preprocessed_1h_master_emeter_20250501_1031_snu_02.csv"

start = pd.Timestamp("2025-05-01 00:00:00")
end   = pd.Timestamp("2025-10-31 23:00:00")
master_index = pd.date_range(start=start, end=end, freq="1H")

# 사용자 정의 휴일(날짜만)
HOLIDAY_LIST_TEXT_2025 = """
2025-01-01,수,신정,법정공휴일
2025-01-28,화,설날연휴,법정공휴일
2025-01-29,수,설날,법정공휴일
2025-01-30,목,설날연휴,법정공휴일
2025-03-01,토,삼일절,법정공휴일
2025-03-03,월,대체공휴일,삼일절 대체공휴일
2025-05-01,목,근로자의날,근로자 휴일
2025-05-05,월,어린이날/부처님오신날,법정공휴일(겹침)
2025-05-06,화,대체공휴일,어린이날/부처님오신날 대체
2025-06-06,금,현충일,법정공휴일
2025-08-15,금,광복절,법정공휴일
2025-10-03,금,개천절,법정공휴일
2025-10-05,일,추석연휴,법정공휴일
2025-10-06,월,추석,법정공휴일
2025-10-07,화,추석연휴,법정공휴일
2025-10-08,수,대체공휴일,추석 대체공휴일
2025-10-09,목,한글날,법정공휴일
2025-12-25,목,크리스마스,법정공휴일
""".strip()

def parse_holiday_dates(holiday_text: str) -> set:
    """
    holiday_text에서 첫 번째 컬럼(YYYY-MM-DD)만 파싱하여 date set으로 반환.
    """
    dates = []
    for line in holiday_text.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 1:
            continue
        d = pd.to_datetime(parts[0], errors="coerce")
        if pd.isna(d):
            continue
        dates.append(d.date())
    return set(dates)

HOLIDAY_DATES_2025 = parse_holiday_dates(HOLIDAY_LIST_TEXT_2025)



# ============================================================
# 1) 파일 이름에서 날짜(YYYYMMDD) 뽑기
# ============================================================
def extract_date_from_filename(filename: str) -> pd.Timestamp:
    m = re.search(r"(\d{8})", filename)
    if not m:
        raise ValueError(f"파일명에서 날짜(YYYYMMDD)를 찾을 수 없습니다: {filename}")
    return pd.to_datetime(m.group(1), format="%Y%m%d")

# ============================================================
# 2) Time 컬럼과 파일명 날짜로 datetime 만들기
# ============================================================
def make_datetime_from_time_and_filename(df: pd.DataFrame, file_path: Path) -> pd.DataFrame:
    if "Time" not in df.columns:
        raise ValueError(f"'Time' 컬럼이 없습니다. 파일: {file_path}")

    file_date = extract_date_from_filename(file_path.name)

    df = df.copy()
    df["datetime"] = pd.to_datetime(
        file_date.strftime("%Y-%m-%d") + " " + df["Time"].astype(str),
        errors="coerce"
    )
    df = df.dropna(subset=["datetime"])
    df = df.sort_values("datetime").set_index("datetime")
    return df

# ============================================================
# 3) NULL(\x00) 제거 후 read_csv
# ============================================================
def read_csv_remove_nulls(path: Path) -> pd.DataFrame:
    with open(path, "rb") as fh:
        raw = fh.read()
    if b"\x00" in raw:
        print(f"[WARN] NULL 바이트 감지 → 제거 후 로드: {path.name}")
        raw = raw.replace(b"\x00", b"")
    return pd.read_csv(io.BytesIO(raw))

# ============================================================
# 4) 폴더 안 CSV를 모두 읽어서 concat
# ============================================================
def load_all_csv_time_from_filename(folder: Path, pattern: str) -> pd.DataFrame | None:
    files = sorted(folder.glob(pattern))
    if not files:
        print(f"[WARN] {folder} 에 {pattern}에 해당하는 파일이 없습니다.")
        return None

    dfs = []
    for f in files:
        try:
            df_raw = read_csv_remove_nulls(f)
            df = make_datetime_from_time_and_filename(df_raw, f)
            df["src_file"] = f.name
            dfs.append(df)
        except Exception as e:
            print(f"[ERROR] {f} 로드 실패: {e}")

    if not dfs:
        return None

    return pd.concat(dfs, axis=0).sort_index()

# ============================================================
# 5) 시간 feature 생성 (1시간 기준)
#    - is_holiday
#    - day_cos/sin: 24-step
#    - week_cos/sin: 168-step
# ============================================================
def create_time_features_1h(index: pd.DatetimeIndex,
                            holiday_dates: set) -> pd.DataFrame:
    df_time = pd.DataFrame(index=index)

    # 토/일 여부: Mon=0 ... Sun=6
    is_weekend = (df_time.index.weekday >= 5)

    # 공휴일 리스트 포함 여부
    dates = df_time.index.date
    is_listed_holiday = pd.Series(dates, index=df_time.index).isin(holiday_dates).values

    # 최종 휴일 플래그
    df_time["is_holiday"] = (is_weekend | is_listed_holiday).astype(int)

    # 24-step cycle
    hour_of_day = df_time.index.hour.astype(int)  # 0~23
    df_time["day_sin"] = np.sin(2 * np.pi * hour_of_day / 24.0)
    df_time["day_cos"] = np.cos(2 * np.pi * hour_of_day / 24.0)

    # 168-step cycle (start 기준으로 시간 인덱스)
    k = np.arange(len(df_time), dtype=float)
    df_time["week_sin"] = np.sin(2 * np.pi * k / 168.0)
    df_time["week_cos"] = np.cos(2 * np.pi * k / 168.0)

    return df_time
# ============================================================
# 6) EMETER 전처리 (10분단위 c_real → 1시간 합)
#    - slot: ceil("1H") 사용 (창 끝 시각)
#    - slot별 c_real 합 = Power_1h
#    - 어떤 slot에 row가 없으면 Power_1h = 0
#
# 예)
#  target slot = 2025-10-07 12:00
#  emeter rows: 11:20:15, 11:30:15만 존재
#  => 둘 다 ceil -> 12:00 슬롯에 들어가므로 합쳐짐
# ============================================================
def preprocess_emeter_power_1h(df_emeter: pd.DataFrame,
                              freq: str = "1H",
                              power_col: str = "c_real") -> pd.DataFrame:
    # 컬럼명 대소문자/변형 대응
    cols_lower = {c.lower(): c for c in df_emeter.columns}
    if power_col.lower() not in cols_lower:
        raise ValueError(f"EMETER에 '{power_col}' 컬럼이 없습니다. 현재 컬럼: {df_emeter.columns.tolist()}")
    P_COL = cols_lower[power_col.lower()]

    df = df_emeter.copy()
    df[P_COL] = pd.to_numeric(df[P_COL], errors="coerce").fillna(0.0)

    # slot (창 끝)
    df["slot"] = df.index.ceil(freq)

    # slot별 합
    p_sum = df.groupby("slot")[P_COL].sum()

    # master timeline에 맞춰: 없는 slot은 0
    out = pd.DataFrame({"Power_1h": p_sum}).reindex(master_index).fillna(0.0)
    return out

# ============================================================
# 7) 전체 파이프라인
# ============================================================
def main():
    # 1) 시간 feature
    df_time = create_time_features_1h(master_index, holiday_dates=HOLIDAY_DATES_2025)

    # --------------------------------------------------------
    # 2) EMETER 로드 & 슬라이싱
    # --------------------------------------------------------
    # TODO: 파일 패턴을 실제 파일명에 맞게 수정 (예: "DBG_EMETER_*.csv")
    df_emeter_raw = load_all_csv_time_from_filename(EMETER_DIR, "*.csv")
    if df_emeter_raw is None:
        raise RuntimeError("EMETER 로그를 읽지 못했습니다. 경로/패턴을 확인하세요.")
    print("[INFO] EMETER raw shape:", df_emeter_raw.shape)

    raw_start = start - pd.Timedelta("1H")
    df_emeter_raw = df_emeter_raw.loc[(df_emeter_raw.index >= raw_start) & (df_emeter_raw.index <= end)]
    print("[INFO] EMETER clipped shape:", df_emeter_raw.shape)

    power_1h = preprocess_emeter_power_1h(df_emeter_raw, freq="1H", power_col="c_real")
    print("[INFO] EMETER 1H shape:", power_1h.shape)

    # --------------------------------------------------------
    # 3) merge (요청: day/week sincos + holiday + power)
    # --------------------------------------------------------
    df_all = df_time.join(power_1h, how="left")
    df_all["Power_1h"] = df_all["Power_1h"].fillna(0.0)

    print("[INFO] Final shape:", df_all.shape)
    print(df_all.head(3))
    print(df_all.tail(3))

    df_all.to_csv(OUT_CSV, index_label="datetime")
    print(f"[INFO] Saved → {OUT_CSV}")

if __name__ == "__main__":
    main()


서울대 3층 25년 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import io

# ============================================================
# 0) 기본 설정 (1시간 기준)
# ============================================================
EMETER_DIR = Path("/home/hd/Desktop/LOG_EMETER_snu")   # TODO: 실제 폴더로 수정
OUT_CSV    = "./preprocessed_1h_master_emeter_20250501_1031_snu.csv"

start = pd.Timestamp("2025-05-01 00:00:00")
end   = pd.Timestamp("2025-10-31 23:00:00")
master_index = pd.date_range(start=start, end=end, freq="1H")

# 사용자 정의 휴일(날짜만)
HOLIDAY_LIST_TEXT_2025 = """
2025-01-01,수,신정,법정공휴일
2025-01-28,화,설날연휴,법정공휴일
2025-01-29,수,설날,법정공휴일
2025-01-30,목,설날연휴,법정공휴일
2025-03-01,토,삼일절,법정공휴일
2025-03-03,월,대체공휴일,삼일절 대체공휴일
2025-05-01,목,근로자의날,근로자 휴일
2025-05-05,월,어린이날/부처님오신날,법정공휴일(겹침)
2025-05-06,화,대체공휴일,어린이날/부처님오신날 대체
2025-06-06,금,현충일,법정공휴일
2025-08-15,금,광복절,법정공휴일
2025-10-03,금,개천절,법정공휴일
2025-10-05,일,추석연휴,법정공휴일
2025-10-06,월,추석,법정공휴일
2025-10-07,화,추석연휴,법정공휴일
2025-10-08,수,대체공휴일,추석 대체공휴일
2025-10-09,목,한글날,법정공휴일
2025-12-25,목,크리스마스,법정공휴일
""".strip()

def parse_holiday_dates(holiday_text: str) -> set:
    """
    holiday_text에서 첫 번째 컬럼(YYYY-MM-DD)만 파싱하여 date set으로 반환.
    """
    dates = []
    for line in holiday_text.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 1:
            continue
        d = pd.to_datetime(parts[0], errors="coerce")
        if pd.isna(d):
            continue
        dates.append(d.date())
    return set(dates)

HOLIDAY_DATES_2025 = parse_holiday_dates(HOLIDAY_LIST_TEXT_2025)



# ============================================================
# 1) 파일 이름에서 날짜(YYYYMMDD) 뽑기
# ============================================================
def extract_date_from_filename(filename: str) -> pd.Timestamp:
    m = re.search(r"(\d{8})", filename)
    if not m:
        raise ValueError(f"파일명에서 날짜(YYYYMMDD)를 찾을 수 없습니다: {filename}")
    return pd.to_datetime(m.group(1), format="%Y%m%d")

# ============================================================
# 2) Time 컬럼과 파일명 날짜로 datetime 만들기
# ============================================================
def make_datetime_from_time_and_filename(df: pd.DataFrame, file_path: Path) -> pd.DataFrame:
    if "Time" not in df.columns:
        raise ValueError(f"'Time' 컬럼이 없습니다. 파일: {file_path}")

    file_date = extract_date_from_filename(file_path.name)

    df = df.copy()
    df["datetime"] = pd.to_datetime(
        file_date.strftime("%Y-%m-%d") + " " + df["Time"].astype(str),
        errors="coerce"
    )
    df = df.dropna(subset=["datetime"])
    df = df.sort_values("datetime").set_index("datetime")
    return df

# ============================================================
# 3) NULL(\x00) 제거 후 read_csv
# ============================================================
def read_csv_remove_nulls(path: Path) -> pd.DataFrame:
    with open(path, "rb") as fh:
        raw = fh.read()
    if b"\x00" in raw:
        print(f"[WARN] NULL 바이트 감지 → 제거 후 로드: {path.name}")
        raw = raw.replace(b"\x00", b"")
    return pd.read_csv(io.BytesIO(raw))

# ============================================================
# 4) 폴더 안 CSV를 모두 읽어서 concat
# ============================================================
def load_all_csv_time_from_filename(folder: Path, pattern: str) -> pd.DataFrame | None:
    files = sorted(folder.glob(pattern))
    if not files:
        print(f"[WARN] {folder} 에 {pattern}에 해당하는 파일이 없습니다.")
        return None

    dfs = []
    for f in files:
        try:
            df_raw = read_csv_remove_nulls(f)
            df = make_datetime_from_time_and_filename(df_raw, f)
            df["src_file"] = f.name
            dfs.append(df)
        except Exception as e:
            print(f"[ERROR] {f} 로드 실패: {e}")

    if not dfs:
        return None

    return pd.concat(dfs, axis=0).sort_index()

# ============================================================
# 5) 시간 feature 생성 (1시간 기준)
#    - is_holiday
#    - day_cos/sin: 24-step
#    - week_cos/sin: 168-step
# ============================================================
def create_time_features_1h(index: pd.DatetimeIndex,
                            holiday_dates: set) -> pd.DataFrame:
    df_time = pd.DataFrame(index=index)

    # 토/일 여부: Mon=0 ... Sun=6
    is_weekend = (df_time.index.weekday >= 5)

    # 공휴일 리스트 포함 여부
    dates = df_time.index.date
    is_listed_holiday = pd.Series(dates, index=df_time.index).isin(holiday_dates).values

    # 최종 휴일 플래그
    df_time["is_holiday"] = (is_weekend | is_listed_holiday).astype(int)

    # 24-step cycle
    hour_of_day = df_time.index.hour.astype(int)  # 0~23
    df_time["day_sin"] = np.sin(2 * np.pi * hour_of_day / 24.0)
    df_time["day_cos"] = np.cos(2 * np.pi * hour_of_day / 24.0)

    # 168-step cycle (start 기준으로 시간 인덱스)
    k = np.arange(len(df_time), dtype=float)
    df_time["week_sin"] = np.sin(2 * np.pi * k / 168.0)
    df_time["week_cos"] = np.cos(2 * np.pi * k / 168.0)

    return df_time
# ============================================================
# 6) EMETER 전처리 (10분단위 c_real → 1시간 합)
#    - slot: ceil("1H") 사용 (창 끝 시각)
#    - slot별 c_real 합 = Power_1h
#    - 어떤 slot에 row가 없으면 Power_1h = 0
#
# 예)
#  target slot = 2025-10-07 12:00
#  emeter rows: 11:20:15, 11:30:15만 존재
#  => 둘 다 ceil -> 12:00 슬롯에 들어가므로 합쳐짐
# ============================================================
def preprocess_emeter_power_1h(df_emeter: pd.DataFrame,
                              freq: str = "1H",
                              power_col: str = "c_real") -> pd.DataFrame:
    # 컬럼명 대소문자/변형 대응
    cols_lower = {c.lower(): c for c in df_emeter.columns}
    if power_col.lower() not in cols_lower:
        raise ValueError(f"EMETER에 '{power_col}' 컬럼이 없습니다. 현재 컬럼: {df_emeter.columns.tolist()}")
    P_COL = cols_lower[power_col.lower()]

    df = df_emeter.copy()
    df[P_COL] = pd.to_numeric(df[P_COL], errors="coerce").fillna(0.0)

    # slot (창 끝)
    df["slot"] = df.index.ceil(freq)

    # slot별 합
    p_sum = df.groupby("slot")[P_COL].sum()

    # master timeline에 맞춰: 없는 slot은 0
    out = pd.DataFrame({"Power_1h": p_sum}).reindex(master_index).fillna(0.0)
    return out

# ============================================================
# 7) 전체 파이프라인
# ============================================================
def main():
    # 1) 시간 feature
    df_time = create_time_features_1h(master_index, holiday_dates=HOLIDAY_DATES_2025)

    # --------------------------------------------------------
    # 2) EMETER 로드 & 슬라이싱
    # --------------------------------------------------------
    # TODO: 파일 패턴을 실제 파일명에 맞게 수정 (예: "DBG_EMETER_*.csv")
    df_emeter_raw = load_all_csv_time_from_filename(EMETER_DIR, "*.csv")
    if df_emeter_raw is None:
        raise RuntimeError("EMETER 로그를 읽지 못했습니다. 경로/패턴을 확인하세요.")
    print("[INFO] EMETER raw shape:", df_emeter_raw.shape)

    raw_start = start - pd.Timedelta("1H")
    df_emeter_raw = df_emeter_raw.loc[(df_emeter_raw.index >= raw_start) & (df_emeter_raw.index <= end)]
    print("[INFO] EMETER clipped shape:", df_emeter_raw.shape)

    power_1h = preprocess_emeter_power_1h(df_emeter_raw, freq="1H", power_col="c_real")
    print("[INFO] EMETER 1H shape:", power_1h.shape)

    # --------------------------------------------------------
    # 3) merge (요청: day/week sincos + holiday + power)
    # --------------------------------------------------------
    df_all = df_time.join(power_1h, how="left")
    df_all["Power_1h"] = df_all["Power_1h"].fillna(0.0)

    print("[INFO] Final shape:", df_all.shape)
    print(df_all.head(3))
    print(df_all.tail(3))

    df_all.to_csv(OUT_CSV, index_label="datetime")
    print(f"[INFO] Saved → {OUT_CSV}")

if __name__ == "__main__":
    main()


서울대 3층 24년 

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
import io

# ============================================================
# 0) 기본 설정 (1시간 기준)
# ============================================================
EMETER_DIR = Path("/home/hd/Desktop/LOG_EMETER_snu")   # TODO: 실제 폴더로 수정
OUT_CSV    = "./preprocessed_1h_master_emeter_20240501_1031_snu.csv"

start = pd.Timestamp("2024-05-01 00:00:00")
end   = pd.Timestamp("2024-10-31 23:00:00")
master_index = pd.date_range(start=start, end=end, freq="1H")

# 사용자 정의 휴일(날짜만)
HOLIDAY_LIST_TEXT_2025 = """
2024-01-01,월,신정,공휴일
2024-02-09,금,설날 연휴,공휴일
2024-02-10,토,설날,공휴일
2024-02-11,일,설날 연휴,공휴일
2024-02-12,월,대체공휴일,설날 대체공휴일
2024-03-01,금,삼일절,공휴일
2024-04-10,수,제22대 국회의원 선거,공휴일
2024-05-01,수,근로자의 날,기념일(일부 휴무)
2024-05-05,일,어린이날,공휴일
2024-05-06,월,대체공휴일,어린이날 대체공휴일
2024-05-15,수,부처님오신날,공휴일
2024-06-06,목,현충일,공휴일
2024-08-15,목,광복절,공휴일
2024-09-16,월,추석 연휴,공휴일
2024-09-17,화,추석,공휴일
2024-09-18,수,추석 연휴,공휴일
2024-10-01,화,국군의 날,임시공휴일(2024 한정)
2024-10-03,목,개천절,공휴일
2024-10-09,수,한글날,공휴일
2024-12-25,수,기독탄신일(성탄절),공휴일
""".strip()

def parse_holiday_dates(holiday_text: str) -> set:
    """
    holiday_text에서 첫 번째 컬럼(YYYY-MM-DD)만 파싱하여 date set으로 반환.
    """
    dates = []
    for line in holiday_text.splitlines():
        line = line.strip()
        if not line:
            continue
        parts = [p.strip() for p in line.split(",")]
        if len(parts) < 1:
            continue
        d = pd.to_datetime(parts[0], errors="coerce")
        if pd.isna(d):
            continue
        dates.append(d.date())
    return set(dates)

HOLIDAY_DATES_2025 = parse_holiday_dates(HOLIDAY_LIST_TEXT_2025)



# ============================================================
# 1) 파일 이름에서 날짜(YYYYMMDD) 뽑기
# ============================================================
def extract_date_from_filename(filename: str) -> pd.Timestamp:
    m = re.search(r"(\d{8})", filename)
    if not m:
        raise ValueError(f"파일명에서 날짜(YYYYMMDD)를 찾을 수 없습니다: {filename}")
    return pd.to_datetime(m.group(1), format="%Y%m%d")

# ============================================================
# 2) Time 컬럼과 파일명 날짜로 datetime 만들기
# ============================================================
def make_datetime_from_time_and_filename(df: pd.DataFrame, file_path: Path) -> pd.DataFrame:
    if "Time" not in df.columns:
        raise ValueError(f"'Time' 컬럼이 없습니다. 파일: {file_path}")

    file_date = extract_date_from_filename(file_path.name)

    df = df.copy()
    df["datetime"] = pd.to_datetime(
        file_date.strftime("%Y-%m-%d") + " " + df["Time"].astype(str),
        errors="coerce"
    )
    df = df.dropna(subset=["datetime"])
    df = df.sort_values("datetime").set_index("datetime")
    return df

# ============================================================
# 3) NULL(\x00) 제거 후 read_csv
# ============================================================
def read_csv_remove_nulls(path: Path) -> pd.DataFrame:
    with open(path, "rb") as fh:
        raw = fh.read()
    if b"\x00" in raw:
        print(f"[WARN] NULL 바이트 감지 → 제거 후 로드: {path.name}")
        raw = raw.replace(b"\x00", b"")
    return pd.read_csv(io.BytesIO(raw))

# ============================================================
# 4) 폴더 안 CSV를 모두 읽어서 concat
# ============================================================
def load_all_csv_time_from_filename(folder: Path, pattern: str) -> pd.DataFrame | None:
    files = sorted(folder.glob(pattern))
    if not files:
        print(f"[WARN] {folder} 에 {pattern}에 해당하는 파일이 없습니다.")
        return None

    dfs = []
    for f in files:
        try:
            df_raw = read_csv_remove_nulls(f)
            df = make_datetime_from_time_and_filename(df_raw, f)
            df["src_file"] = f.name
            dfs.append(df)
        except Exception as e:
            print(f"[ERROR] {f} 로드 실패: {e}")

    if not dfs:
        return None

    return pd.concat(dfs, axis=0).sort_index()

# ============================================================
# 5) 시간 feature 생성 (1시간 기준)
#    - is_holiday
#    - day_cos/sin: 24-step
#    - week_cos/sin: 168-step
# ============================================================
def create_time_features_1h(index: pd.DatetimeIndex,
                            holiday_dates: set) -> pd.DataFrame:
    df_time = pd.DataFrame(index=index)

    # 토/일 여부: Mon=0 ... Sun=6
    is_weekend = (df_time.index.weekday >= 5)

    # 공휴일 리스트 포함 여부
    dates = df_time.index.date
    is_listed_holiday = pd.Series(dates, index=df_time.index).isin(holiday_dates).values

    # 최종 휴일 플래그
    df_time["is_holiday"] = (is_weekend | is_listed_holiday).astype(int)

    # 24-step cycle
    hour_of_day = df_time.index.hour.astype(int)  # 0~23
    df_time["day_sin"] = np.sin(2 * np.pi * hour_of_day / 24.0)
    df_time["day_cos"] = np.cos(2 * np.pi * hour_of_day / 24.0)

    # 168-step cycle (start 기준으로 시간 인덱스)
    k = np.arange(len(df_time), dtype=float)
    df_time["week_sin"] = np.sin(2 * np.pi * k / 168.0)
    df_time["week_cos"] = np.cos(2 * np.pi * k / 168.0)

    return df_time
# ============================================================
# 6) EMETER 전처리 (10분단위 c_real → 1시간 합)
#    - slot: ceil("1H") 사용 (창 끝 시각)
#    - slot별 c_real 합 = Power_1h
#    - 어떤 slot에 row가 없으면 Power_1h = 0
#
# 예)
#  target slot = 2025-10-07 12:00
#  emeter rows: 11:20:15, 11:30:15만 존재
#  => 둘 다 ceil -> 12:00 슬롯에 들어가므로 합쳐짐
# ============================================================
def preprocess_emeter_power_1h(df_emeter: pd.DataFrame,
                              freq: str = "1H",
                              power_col: str = "c_real") -> pd.DataFrame:
    # 컬럼명 대소문자/변형 대응
    cols_lower = {c.lower(): c for c in df_emeter.columns}
    if power_col.lower() not in cols_lower:
        raise ValueError(f"EMETER에 '{power_col}' 컬럼이 없습니다. 현재 컬럼: {df_emeter.columns.tolist()}")
    P_COL = cols_lower[power_col.lower()]

    df = df_emeter.copy()
    df[P_COL] = pd.to_numeric(df[P_COL], errors="coerce").fillna(0.0)

    # slot (창 끝)
    df["slot"] = df.index.ceil(freq)

    # slot별 합
    p_sum = df.groupby("slot")[P_COL].sum()

    # master timeline에 맞춰: 없는 slot은 0
    out = pd.DataFrame({"Power_1h": p_sum}).reindex(master_index).fillna(0.0)
    return out

# ============================================================
# 7) 전체 파이프라인
# ============================================================
def main():
    # 1) 시간 feature
    df_time = create_time_features_1h(master_index, holiday_dates=HOLIDAY_DATES_2025)

    # --------------------------------------------------------
    # 2) EMETER 로드 & 슬라이싱
    # --------------------------------------------------------
    # TODO: 파일 패턴을 실제 파일명에 맞게 수정 (예: "DBG_EMETER_*.csv")
    df_emeter_raw = load_all_csv_time_from_filename(EMETER_DIR, "*.csv")
    if df_emeter_raw is None:
        raise RuntimeError("EMETER 로그를 읽지 못했습니다. 경로/패턴을 확인하세요.")
    print("[INFO] EMETER raw shape:", df_emeter_raw.shape)

    raw_start = start - pd.Timedelta("1H")
    df_emeter_raw = df_emeter_raw.loc[(df_emeter_raw.index >= raw_start) & (df_emeter_raw.index <= end)]
    print("[INFO] EMETER clipped shape:", df_emeter_raw.shape)

    power_1h = preprocess_emeter_power_1h(df_emeter_raw, freq="1H", power_col="c_real")
    print("[INFO] EMETER 1H shape:", power_1h.shape)

    # --------------------------------------------------------
    # 3) merge (요청: day/week sincos + holiday + power)
    # --------------------------------------------------------
    df_all = df_time.join(power_1h, how="left")
    df_all["Power_1h"] = df_all["Power_1h"].fillna(0.0)

    print("[INFO] Final shape:", df_all.shape)
    print(df_all.head(3))
    print(df_all.tail(3))

    df_all.to_csv(OUT_CSV, index_label="datetime")
    print(f"[INFO] Saved → {OUT_CSV}")

if __name__ == "__main__":
    main()


플롯해보기 (파일 이름이랑 연도만 바꾸면 됨)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

CSV_PATH = "./preprocessed_1h_master_emeter_20250501_1031_snu.csv"

# 1) 로드
df = pd.read_csv(CSV_PATH, parse_dates=["datetime"])
df = df.set_index("datetime").sort_index()

# 2) 사용할 월 목록 (2025-05 ~ 2025-10)
months = pd.period_range("2025-05", "2025-10", freq="M")

# 3) 월별로 길게 플롯
for p in months:
    month_str = p.strftime("%Y-%m")   # "2025-05"

    # ✅ 반드시 .loc 사용
    df_m = df.loc[month_str]

    plt.figure()
    plt.plot(df_m.index, df_m["Power_1h"].values)
    plt.title(f"Power_1h - {month_str}")
    plt.xlabel("datetime")
    plt.ylabel("Power_1h")
    plt.tight_layout()
    plt.show()
